# LeNet in PyTorch
- Minh Nguyen
- 12/3/2024
- About: 
    - Simple architecture with 2 convolutional layers followed by a fully connected layer
    - It is light-weight, low memory requirement
- Key Features:
    - Input-size: 28x28, single channel
    - 2 Conv Layers: Extract spatial features using 5 x 5 kernels.
    - 2 Fully Connected Layers: Classification after feature extraction
    - Activation layer: Tanh
    - Pooling: average pooling for spatial reduction.
    - Use Case: small dataset

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the LeNet-5 model
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)  # Input: 1x28x28, Output: 6x28x28
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)           # Input: 6x28x28, Output: 16x10x10
        self.fc1 = nn.Linear(16 * 5 * 5, 120)                            # Input: Flattened 16x5x5, Output: 120
        self.fc2 = nn.Linear(120, 84)                                   # Input: 120, Output: 84
        self.fc3 = nn.Linear(84, 10)                                    # Input: 84, Output: 10 (Classes)

    def forward(self, x):
        x = torch.tanh(self.conv1(x))                                   # Apply Tanh activation
        x = nn.functional.avg_pool2d(x, kernel_size=2, stride=2)        # Apply Average Pooling: Output: 6x14x14
        x = torch.tanh(self.conv2(x))                                   # Apply Tanh activation
        x = nn.functional.avg_pool2d(x, kernel_size=2, stride=2)        # Apply Average Pooling: Output: 16x5x5
        x = x.view(-1, 16 * 5 * 5)                                      # Flatten
        x = torch.tanh(self.fc1(x))                                     # Fully connected layer with Tanh
        x = torch.tanh(self.fc2(x))                                     # Fully connected layer with Tanh
        x = self.fc3(x)                                                 # Fully connected layer (no activation for logits)
        return x

# Prepare the MNIST dataset
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize between [-1, 1]
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define training settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Testing Loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Epoch 1/10, Loss: 0.2647
Epoch 2/10, Loss: 0.0843
Epoch 3/10, Loss: 0.0590
Epoch 4/10, Loss: 0.0456
Epoch 5/10, Loss: 0.0369
Epoch 6/10, Loss: 0.0308
Epoch 7/10, Loss: 0.0264
Epoch 8/10, Loss: 0.0242
Epoch 9/10, Loss: 0.0185
Epoch 10/10, Loss: 0.0173
Test Accuracy: 98.65%
